In [1]:
%load_ext nvcc4jupyter

from nvcc4jupyter import set_defaults
set_defaults(compiler_args='-arch=sm_100a -Xptxas=-v')

Source files will be saved in "/tmp/tmpj16ux_ig".


# GOD I AM SO HAPPY, SO HAPPY, THAT JUPYTER IN REMOTE SSH IS WORKING AGAIN! LETS DO SOME SMEM STUFF BABYYYYYY. 

In [34]:
%%cuda

#include<stdio.h> 
#include<stdlib.h>
#include<cuda.h>
#include<cuda_runtime.h> 

constexpr int N_iter = 100000;
constexpr int N_warps = 8;
constexpr int N_lanes = 32;
constexpr int N_blocks = 1;

__global__ void smem_no_bc()
{
  __shared__ float A[N_warps][N_lanes];
  int warp_id = threadIdx.y; 
  int lane_id = threadIdx.x; 
  
  float* ptr = &A[warp_id][lane_id];
  int addr = (int)ptr & 0xFFFF; //#0x is just a prefix meaning its hexadecimal 
  //# each F is a 15, and each digit is worth that power of 16. 
  //# in fact this number is 2^16 + 1, which means we are bit masking with the and 
  //# and extracting the first 16 digits of our pointer. but why? 
  /*
  """
  well really, each address is worth a byte (8 bits) and so how many bytes do we have in this shared memory?
  we have 8*32 bytes, so we have 8*8*32 bits, so its enough for our adress to be that big 
  well 8*8*32 = 2^6 * 2^5 = 2^11, 
  
  but the adress itself, if you do addr + 1, you are moving 8 bits (or 1 byte) 
  so then, if we have 2^16 - 1 addrs, we are guuci, but idk whats more to this nvm
  """
  */
  float r1;
  
  for (int j = 0; j < N_iter; j++) {
      asm volatile ("ld.volatile.shared.f32 %0, [%1];"
                      : "=f"(r1)
                      : "r"(addr));
  }
  
  //#__syncthreads(); (does this make things slow??)
  
}

int main()
{
   
  cudaEvent_t start, stop;
  cudaEventCreate(&start);
  cudaEventCreate(&stop);

 
  cudaEventRecord(start, 0);

 
  //#weirdly enough, it is actually threadidx.x that has stride one, even tho its 
  //#on the left of the tuple. so we gotta be careful about that launch
  dim3 threads(N_lanes, N_warps);
  dim3 grid(N_blocks);
  
  smem_no_bc<<<grid, threads>>>();


  cudaEventRecord(stop, 0);
  cudaEventSynchronize(stop);

  float milliseconds = 0;
  cudaEventElapsedTime(&milliseconds, start, stop);
  printf("GPU Kernel time: %f ms\n", milliseconds);
  
  
  /*
  """
  so how do we calculate our memory transfer speed, well, we have N_warps warps, 
  so per iteration, we issue N_warp loads per bank, so our speed will be 
  N_loads/bank/second. 
  so for N_iter iterations, 
  smem_bank_speed = (N_warps*N_iter)/time 
  """
  */
  float seconds = milliseconds*0.001;
  float mega = 1000000.0;
  printf("mega_loads/bank/second = %f \n", (N_warps*N_iter)/(seconds*mega));
  
  int deviceId;
  cudaGetDevice(&deviceId);
  
  cudaDeviceProp prop;
  cudaGetDeviceProperties(&prop, deviceId);
  

  double smClockMHz = prop.clockRate / 1000.0;
  
  printf("Device: %s\n", prop.name);
  printf("Base SM Clock Rate: %.2f MHz\n", smClockMHz);
}

GPU Kernel time: 0.752800 ms
mega_loads/bank/second = 1062.699219 
Device: NVIDIA GeForce RTX 5060 Ti
Base SM Clock Rate: 2602.00 MHz



In [7]:
import math 
print(math.log((8*8*32), 2))

11.0
